In [2]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np

2023-08-05 16:12:53.225674: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
class CustomDenseLayer(layers.Layer):
    def __init__(self, ao_vals, activation=None,
                 name='matact', **kwargs):
        super().__init__(name=name, **kwargs)
        self.activation = activation
        self.ao_vals = ao_vals
        
    def build(self, shape):
        self.weight = self.add_weight("weight", shape=(self.ao_vals, self.ao_vals), initializer='ones', trainable=True)
        self.bias = self.add_weight("bias", shape=(self.ao_vals,), initializer='zeros', trainable=True)

    def call(self, input):
        out = input
        #out = tf.matmul(out, self.weight)
        out = tf.einsum("nga,ma->ngm", input, self.weight)
        out = out + self.bias
        if self.activation is not None:
            out = self.activation(out)
        return out

def create_model(n, m):
    model = tf.keras.Sequential([
        CustomDenseLayer(m),
        tf.keras.layers.Input(shape=(n, m)),  # Input layer with shape (None, n, m)
        tf.keras.layers.Flatten(),           # Flatten the input to a 1D array
        tf.keras.layers.Dense(64, activation='relu'), # A dense layer with 64 units and ReLU activation
        tf.keras.layers.Dense(1)             # Output layer with 1 unit (the output shape will be ())
    ])
    return model

# Define the input shape
batch_size = 32
n, m = 10, 5

# Generate random training data and labels
train_input_data = np.random.rand(1000, n, m)
train_labels = np.sum(train_input_data, axis=(1, 2))  # Sum along the n and m axes to get a scalar output

# Generate random test data and labels
test_input_data = np.random.rand(200, n, m)
test_labels = np.sum(test_input_data, axis=(1, 2))

# Create the model
model = create_model(n, m)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(train_input_data, train_labels, batch_size=batch_size, epochs=10, validation_split=0.1)

# Evaluate the model on the test data
loss = model.evaluate(test_input_data, test_labels)
model.trainable_weights

Epoch 1/10
29/29 [==============================] - 0s 5ms/step - loss: 229.9398 - val_loss: 22.1219
Epoch 2/10
29/29 [==============================] - 0s 2ms/step - loss: 4.2628 - val_loss: 1.8027
Epoch 3/10
29/29 [==============================] - 0s 2ms/step - loss: 0.5701 - val_loss: 0.4637
Epoch 4/10
29/29 [==============================] - 0s 2ms/step - loss: 0.3289 - val_loss: 0.3405
Epoch 5/10
29/29 [==============================] - 0s 3ms/step - loss: 0.3026 - val_loss: 0.3188
Epoch 6/10
29/29 [==============================] - 0s 3ms/step - loss: 0.2595 - val_loss: 0.2649
Epoch 7/10
29/29 [==============================] - 0s 2ms/step - loss: 0.1912 - val_loss: 0.1562
Epoch 8/10
29/29 [==============================] - 0s 3ms/step - loss: 0.1257 - val_loss: 0.1111
Epoch 9/10
29/29 [==============================] - 0s 2ms/step - loss: 0.0834 - val_loss: 0.0679
Epoch 10/10
7/7 [==============================] - 0s 1ms/step - loss: 0.0408


[<tf.Variable 'matact/weight:0' shape=(5, 5) dtype=float32, numpy=
 array([[1.0328165 , 1.0331463 , 1.033145  , 1.0333078 , 1.0328301 ],
        [1.035669  , 1.0355698 , 1.035665  , 1.0352805 , 1.035592  ],
        [1.0274438 , 1.0304205 , 1.0281752 , 1.0282203 , 1.0285465 ],
        [0.99451655, 0.9935861 , 0.9915516 , 0.99278283, 0.9928436 ],
        [1.0359485 , 1.0354744 , 1.0358444 , 1.0355868 , 1.0362161 ]],
       dtype=float32)>,
 <tf.Variable 'matact/bias:0' shape=(5,) dtype=float32, numpy=
 array([ 0.03404956,  0.0349526 ,  0.03043637, -0.00128854,  0.03506564],
       dtype=float32)>,
 <tf.Variable 'dense_12/kernel:0' shape=(50, 64) dtype=float32, numpy=
 array([[-0.18617716,  0.04211435, -0.10364488, ..., -0.02565217,
         -0.13210194, -0.12227466],
        [ 0.1550087 , -0.1583499 ,  0.03107181, ..., -0.17793581,
         -0.17157532,  0.09623014],
        [ 0.17036697,  0.19806437,  0.1601436 , ...,  0.16820456,
         -0.04039891, -0.03350386],
        ...,
       